# Quantum Prisoner’s Dilemma (EWL) — Notebook

This notebook implements the Eisert–Wilkens–Lewenstein (EWL) quantization of the Prisoner’s Dilemma.
It includes: implementation of the entangling operator J(γ), the general local strategy U(θ,φ),
a `play_game` function using statevector simulation, and visualization of payoffs for several entanglement values γ.

Requirements: `qiskit`, `numpy`, `matplotlib`.

Created on: 2025-11-17 11:24:44 UTC


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, Statevector
plt.rcParams['figure.figsize'] = (6,4)


In [ ]:
def J_matrix(gamma):
    """Return 4x4 matrix for J(gamma) = cos(g/2) I + i sin(g/2) X⊗X"""
    c = np.cos(gamma/2)
    s = np.sin(gamma/2)
    X = np.array([[0,1],[1,0]], dtype=complex)
    I = np.eye(2, dtype=complex)
    XX = np.kron(X, X)
    J = c * np.kron(I, I) + 1j * s * XX
    return J

def U_matrix(theta, phi):
    """Return 2x2 strategy matrix U(theta,phi) as in EWL.
    U = [[e^{i phi} cos(theta/2), sin(theta/2)], [-sin(theta/2), e^{-i phi} cos(theta/2)]]
    """
    a = np.exp(1j * phi) * np.cos(theta/2)
    b = np.sin(theta/2)
    c = -np.sin(theta/2)
    d = np.exp(-1j * phi) * np.cos(theta/2)
    return np.array([[a, b],[c, d]], dtype=complex)

def play_game(theta_A, phi_A, theta_B, phi_B, gamma, payoff_table=None):
    """Play one EWL game using statevector simulation. Returns outcome probabilities and (payoffA, payoffB).
    payoff_table should be dict mapping bitstring to (payoffA, payoffB) for outcomes '00','01','10','11'.
    Qubit order: [A, B] with 0 -> A, 1 -> B when forming kron.
    """
    if payoff_table is None:
        # Standard PD payoffs (R, S, T, P) = (3, 0, 5, 1)
        payoff_table = {
            '00': (3, 3),  # C,C -> both get R
            '01': (0, 5),  # C,D -> A gets S, B gets T
            '10': (5, 0),  # D,C -> A gets T, B gets S
            '11': (1, 1)   # D,D -> both get P
        }
    # Build full unitary: J * (U_A ⊗ U_B) * J^
    J = J_matrix(gamma)
    U_A = U_matrix(theta_A, phi_A)
    U_B = U_matrix(theta_B, phi_B)
    Ufull = np.kron(U_A, U_B)
    # Full op: J * Ufull * J^
    full = J @ Ufull @ np.conjugate(J.T)
    # Apply to |00> state
    psi0 = np.array([1,0,0,0], dtype=complex)
    psi_final = full @ psi0
    # Probabilities
    probs = { '00': np.abs(psi_final[0])**2,
              '01': np.abs(psi_final[1])**2,
              '10': np.abs(psi_final[2])**2,
              '11': np.abs(psi_final[3])**2 }
    payoffA = sum(probs[b] * payoff_table[b][0] for b in probs)
    payoffB = sum(probs[b] * payoff_table[b][1] for b in probs)
    return probs, (payoffA, payoffB)


In [ ]:
# Classical strategies: C = U(0,0) (cooperate), D = U(pi,0) (defect)
theta_C, phi_C = 0.0, 0.0
theta_D, phi_D = np.pi, 0.0
for gamma in [0, np.pi/8, np.pi/4, np.pi/2]:
    p_CC, pay_CC = play_game(theta_C, phi_C, theta_C, phi_C, gamma)
    p_DD, pay_DD = play_game(theta_D, phi_D, theta_D, phi_D, gamma)
    print(f"gamma={gamma:.3f}: C,C pay={pay_CC}, D,D pay={pay_DD}")


In [ ]:
import numpy as np
thetas = np.linspace(0, np.pi, 201)
gammas = [0.0, np.pi/8, np.pi/4, np.pi/2]
results = {}
for gamma in gammas:
    payoffs_A = []
    payoffs_B = []
    for th in thetas:
        _, (pA, pB) = play_game(th, 0.0, th, 0.0, gamma)
        payoffs_A.append(pA)
        payoffs_B.append(pB)
    results[gamma] = (np.array(payoffs_A), np.array(payoffs_B))
print('Computed symmetric-payoff curves for gamma values')


In [ ]:
plt.figure()
for gamma in results:
    pa, pb = results[gamma]
    plt.plot(thetas, pa, label=f'gamma={gamma:.3f}')
plt.xlabel('theta (strategy parameter, phi=0)')
plt.ylabel('Payoff (Player A)')
plt.legend()
plt.title('Symmetric strategy payoffs (U=U) for various entanglement gamma')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
thetasA = np.linspace(0, np.pi, 201)
thetasB = np.linspace(0, np.pi, 201)
def payoff_matrix_for_gamma(gamma, phiA=0, phiB=0):
    PA = np.zeros((len(thetasA), len(thetasB)))
    PB = np.zeros_like(PA)
    for i, ta in enumerate(thetasA):
        for j, tb in enumerate(thetasB):
            _, (pA, pB) = play_game(ta, phiA, tb, phiB, gamma)
            PA[i,j] = pA
            PB[i,j] = pB
    return PA, PB

gamma = np.pi/2
PA, PB = payoff_matrix_for_gamma(gamma)
plt.figure(figsize=(8,6))
plt.imshow(PA, extent=[0, np.pi, 0, np.pi], origin='lower', aspect='auto')
plt.colorbar(label='Payoff A')
plt.xlabel('theta_B')
plt.ylabel('theta_A')
plt.title('Payoff A heatmap (gamma=pi/2, phiA=phiB=0)')
plt.show()


In [ ]:
import json
out = {'thetas': thetas.tolist(), 'gammas': [float(g) for g in gammas], 'results': {str(float(g)): results[g][0].tolist() for g in results}}
with open('ewl_results_summary.json','w') as f:
    json.dump(out, f)
print('Saved summary to ewl_results_summary.json')
